In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

In [ ]:
result = pd.read_excel("E:\\Python\\AIpython\\.ipynb_checkpoints\\demo.xlsx",header=None,nrows=2)
# 不加header=None，第一行默认为索引 ; nrows=2表示读几行
print(result)
result.to_pickle('frame_pickle')#相当于save方法，存为二进制格式数据
pd.read_pickle("frame_pickle")#相当于load方法，读取二进制数据文件


pd.read_csv('path',header=None)生成默认列名，
加参数na_values=['NULL']判断那些值为空，还可将其赋值为字典来为各列指定NA标记值，字典的键代表列名
加参数chunksize=10表示分块读取，每块大小为10
pd.read_csv('path',name=['a','b','c'])生成指定列名，再加上index_col=‘a’就把a作为索引
希望多个列做层次化索引，只需把index_col换成列名的列表

In [16]:
#合并数据，merge参数说明P190
df1 = DataFrame({
    "lkey":['b','b','a','c','a','a','b'],
    "data1":range(7)
})

df2 = DataFrame({
    "rkey":['a','b','d'],
    "data2":range(3)
})

pd.merge(df1,df2,left_on='lkey',right_on='rkey',how='outer')
#默认用重叠的列做键,一个字典中包含多个键，合并时输入键列表
# how指定连接方式，默认内连接
# suffixes用于处理重叠列名（注意是名字，不是变量，只是把相同的名字变一下）
df1.join(df2,how='outer')

,lkey,data1,rkey,data2
0,b,0,a,0.0
1,b,1,b,1.0
2,a,2,d,2.0
3,c,3,NaN,NaN
4,a,4,NaN,NaN
5,a,5,NaN,NaN
6,b,6,NaN,NaN


In [15]:
#轴向连接,concat参数说明P198
s1 = Series([1,2],index=['a','b'])
s2 = Series([3,4,5],index=['c','d','e'])
s3 = Series([6,7],index=['f','g'])
result = pd.concat([s1,s2,s3],keys=['one','two','three'])
#默认axies=0，axies=1生成DataFrame（表格型数据结构，既有行索引，也有列索引，可看作series组成的字典）
#参数join=inner生成交集，join_axes指定在其他轴上合并，keys在连接轴上创建层次化索引
result.unstack()


# combine_first用参数对象的数据为调用对象的数据打补丁
df1 = DataFrame({
    'a':[1,np.nan,5,np.nan],
    'b':[np.nan,2,np.nan,6],
    's':range(2,18,4)
})
df1 = DataFrame({
    'a':[1,5,np.nan,np.nan,7],
    'b':[np.nan,2,11,np.nan,6]
})
df1.combine_first(df2)

,a,b,data2,rkey
0,1.0,NaN,0.0,a
1,5.0,2.0,1.0,b
2,NaN,11.0,2.0,d
3,NaN,NaN,NaN,NaN
4,7.0,6.0,NaN,NaN


In [10]:
#重塑
data = DataFrame(np.arange(6).reshape((2,3)),
                 index=pd.Index(['Ohio','Colorado'],name='state'),
                 columns=pd.Index(['one','two','three'],name='number'))
print(data)
print("============================")
print(data.stack(0))#列转行,默认操作最内层，也可传入分层级别编号或名称
#参数咋设没太懂
print("============================")
print(data.stack().unstack())#再转回来，作为旋转轴的级别会成为结果中的最低级别
#长格式转宽格式，a.pivot('a','b','c')a是行名，b是列名，c是填充数据的列名

number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5
state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32
number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5


In [12]:
#移除重复数据
data = DataFrame({'k1':['one']*3+['two']*4,
                  'k2':[1,1,2,3,3,4,4]})
print(data.duplicated())#返回布尔值，判断是否重复
data.drop_duplicates()#删除重复

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool


,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [18]:
#利用函数或映射进行数据转换
data = DataFrame({
    'food':['bacon','pulled pork','bacon','pastrami','corned beef','bacon','pastrami','honey ham','nova lox'],
    'ounces':[4,3,12,6,7.5,8,3,5,6]
})
print(data)
meat_to_animal = {
    'bacon':'pig',
    'pulled pork':'pig',
    'corned beef':'cow',
    'honey ham':'pig',
    'nova lox':'salmon'
}
print("======================")
# map修改对象的数据子集
data['animal'] = data['food'].map(meat_to_animal)
print(data)
#使用rename更新标签
print("======================")
print(data.rename(index={0:'nan'},
            columns={'animal':'ANIMAL'}))

          food  ounces
0        bacon     4.0
1  pulled pork     3.0
2        bacon    12.0
3     pastrami     6.0
4  corned beef     7.5
5        bacon     8.0
6     pastrami     3.0
7    honey ham     5.0
8     nova lox     6.0
          food  ounces  animal
0        bacon     4.0     pig
1  pulled pork     3.0     pig
2        bacon    12.0     pig
3     pastrami     6.0     NaN
4  corned beef     7.5     cow
5        bacon     8.0     pig
6     pastrami     3.0     NaN
7    honey ham     5.0     pig
8     nova lox     6.0  salmon
            food  ounces  ANIMAL
nan        bacon     4.0     pig
1    pulled pork     3.0     pig
2          bacon    12.0     pig
3       pastrami     6.0     NaN
4    corned beef     7.5     cow
5          bacon     8.0     pig
6       pastrami     3.0     NaN
7      honey ham     5.0     pig
8       nova lox     6.0  salmon


In [15]:
#替换值
data = Series([1.,-999,2.,-999,-1000,3])
print(data)
print(data.replace([-999,-1000],[np.nan,0.00001]))
#输入的参数也可以是字典

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64
0    1.00000
1        NaN
2    2.00000
3        NaN
4    0.00001
5    3.00000
dtype: float64


In [8]:
#离散化和面元划分
ages = [20,22,25,27,21,23,37,31,61,45,41,32]
bins = [18,25,35,60,100]#若传入间距数量，会按最大值和最小追等间距分开
group_name = ['a','b','c','d']#4个区间
cat = pd.cut(ages,bins,labels=group_name)#还可以加参数labels=group_name来指定分类名称
print(cat.codes)#相当于cat.labels指定所属类别
print(pd.value_counts(cat))#统计个数

#qcut可根据样本分位数进行划分，也就是每个字区间样本一样多。当然也可以自定义

[0 0 0 1 0 0 2 1 3 2 2 1]
a    5
b    3
c    3
d    1
dtype: int64


In [9]:
#分类变量转换为指标矩阵
data= pd.DataFrame({'key':['b','b','a','c','a','b'],
                    'data1':range(6)})
print(pd.get_dummies(data['key']))

   a  b  c
0  0  1  0
1  0  1  0
2  1  0  0
3  0  0  1
4  1  0  0
5  0  1  0


In [11]:
#字符串对象化
#字符串方法P218
val = 'a,b,   cccp'
piece = [x.strip() for x in val.split(',')]#split分开，strip去空格
print(piece)
#用符号将数据连起来
print("::".join(piece))


['a', 'b', 'cccp']
a::b::cccp
